In [1]:
import pandas as pd
df=pd.read_csv('/data/cluster_travel_matrix.csv')

In [2]:
df.head()

,stopA,stopB,distance,speed_0_0,speed_0_1,speed_0_2,speed_0_3,speed_0_4,speed_0_5,speed_0_6,...,speed_6_14,speed_6_15,speed_6_16,speed_6_17,speed_6_18,speed_6_19,speed_6_20,speed_6_21,speed_6_22,speed_6_23
0,7027,4538,0.553323,0,0,0,0,0.0,0.0,0.006207,...,0.005261,0.005385,0.005113,0.005050,0.004771,0.004913,0.005084,0.005158,0.005142,0.005373
1,7027,1510,1.830883,0,0,0,0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2478,4527,0.236727,0,0,0,0,0.0,0.0,0.003909,...,0.004584,0.005057,0.005174,0.004794,0.004427,0.004819,0.004648,0.005367,0.005450,0.005812
3,2478,1170,0.259647,0,0,0,0,0.0,0.0,0.005031,...,0.005066,0.005121,0.005026,0.004840,0.004844,0.004666,0.004401,0.005370,0.005245,0.006293
4,553,554,0.309288,0,0,0,0,0.0,0.0,0.000000,...,0.014440,0.013880,0.014202,0.013683,0.012563,0.014668,0.014989,0.014697,0.015846,0.016150


In [3]:
import pandas as pd
import numpy as np
df=df.replace([np.inf, -np.inf], np.nan)
df=df.dropna()

In [4]:
from sklearn.cluster import KMeans

In [5]:
kmeans = KMeans(n_clusters=100).fit(df[[f for f in df.columns if f not in ['distance','stopA','stopB']]])

In [7]:
df['labels']=kmeans.labels_

In [8]:
cluster0=df[df['labels']==50]
from dbanalysis import stop_tools
to_concat = []
count=0

for row in cluster0.itertuples():
    print(count)
    fs = str(row[1])
    ts=str(row[2])
    to_concat.append(stop_tools.stop_data(fs,ts))
cluster_stops = pd.concat(to_concat,axis=0)
cluster_stops.to_csv('/data/cluser1.csv',index=False)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [9]:
df = cluster_stops

In [15]:
df['distance'].unique()

array([0.65818432, 0.53991997, 0.29521969, 0.34096893, 0.3183777 ,
       0.38578351, 0.47558529, 0.46517043, 0.80970194, 0.49898211,
       0.3955824 , 0.33707048, 0.56204434, 0.63100014, 0.44608729,
       0.34666323, 0.28733866, 0.19544775, 0.2126229 , 0.28136019,
       0.1664519 , 0.4659546 , 0.23040296, 0.2385152 , 0.36993892,
       0.45092109, 0.31875369, 0.40947728])

In [16]:
features = ['day','hour','distance','vappr','temp']

In [18]:
from sklearn.ensemble import RandomForestRegressor as rf

In [19]:
model = rf().fit(df[features],df['traveltime'])

In [20]:
preds = model.predict(df[features])

In [21]:
from sklearn import metrics

In [22]:
metrics.r2_score(df['traveltime'],preds)

0.8470403061738947

In [23]:
train = df[df['year']==2016]
test = df[df['year']==2017]

In [24]:
model = rf().fit(train[features],train['traveltime'])

In [25]:
preds = model.predict(test[features])

In [26]:
metrics.r2_score(test['traveltime'],preds)

0.30336200617024056

In [27]:
((abs(test['traveltime']-preds)/test['traveltime']) * 100).mean()

33.81305737628167

In [28]:
from sklearn.neural_network import MLPRegressor as mlp
from sklearn.preprocessing import StandardScaler as ss

In [30]:
X_scaler = ss()
X = X_scaler.fit_transform(train[features])
Y_scaler = ss()
Y = Y_scaler.fit_transform(train['traveltime'].values.reshape(-1,1))

/home/student/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [31]:
model = mlp().fit(X,Y)

/home/student/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
X = X_scaler.transform(test[features])
preds = Y_scaler.inverse_transform(model.predict(X))

In [33]:
metrics.r2_score(test['traveltime'],preds)

0.1325654220637731

In [35]:
preds.max()

124.3216653343645

In [36]:
((abs(test['traveltime'] - preds)/test['traveltime'])*100).mean()

50.22619705652563

In [45]:
df.columns

Index(['index', 'dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'fromstop',
       'tostop', 'traveltime', 'dwelltime', 'distance', 'speed', 'dt', 'date',
       'day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb'],
      dtype='object')

In [64]:
to_concat = []
for hour in df['hour'].unique():
    temp = df[df['hour']==hour]
    for stopA in temp['fromstop'].unique():
        temp2 = temp[temp['fromstop']==stopA]
        for stopB in temp2['tostop'].unique():
            temp3 = temp2[temp2['tostop']==stopB]
            temp3['mean_travel_time']=temp3['traveltime'].mean()
            to_concat.append(temp3)
df=pd.concat(to_concat,axis=0)

# Classifying lateness on the cluster

In [47]:
from sklearn.ensemble import RandomForestClassifier as rf

In [65]:
df['late']=df['traveltime']>df['mean_travel_time']
df=df.dropna()

In [18]:
df.columns

Index(['index', 'dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'fromstop',
       'tostop', 'traveltime', 'dwelltime', 'distance', 'speed', 'dt', 'date',
       'day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time', 'late'],
      dtype='object')

In [66]:
train = df[df['year']==2016]
test= df[df['year']==2017]

In [67]:
clf=rf().fit(train[['actualtime_arr_from','day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time']+['distance']],train['late'])

In [68]:
preds = clf.predict(test[['actualtime_arr_from','day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time']+['distance']])

In [69]:
from sklearn.metrics import accuracy_score

In [71]:
accuracy_score(test['late'],preds)

0.5831513295656677

In [53]:
df=cluster_stops
to_concat = []
for hour in df['hour'].unique():
    temp = df[df['hour']==hour]
    temp['mean_travel_time']=temp['traveltime'].mean()
    to_concat.append(temp)
    
            
df=pd.concat(to_concat,axis=0)

/home/student/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [56]:
df['late']=df['traveltime']>df['mean_travel_time']
df=df.dropna()
train = df[df['year']==2016]
test= df[df['year']==2017]

In [72]:
clf=rf().fit(train[['hour', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time']+['distance']],train['late'])

In [73]:
preds = clf.predict(test[['hour','dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time']+['distance']])

In [74]:
accuracy_score(test['late'],preds)

0.5714655618850336

# So we can classify for these larger means (for all stops), but not for the more specific means (just for some stops)

In [75]:
from sklearn.decomposition import PCA

In [78]:
X = PCA(n_components=10).fit_transform(df[['actualtime_arr_from','day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time','distance']])

In [105]:
late = np.array([[x for x in df['late']]])

In [107]:
X=pd.DataFrame(X)

In [110]:
X['late']=df['late']

ValueError: cannot reindex from a duplicate axis

In [95]:
X[0:,11:12]=[i for i in df['late']]

ValueError: could not broadcast input array from shape (228097) into shape (228097,0)

In [84]:
clf=rf().fit(X,df['late'])

In [86]:
preds=clf.predict(X)

In [87]:
accuracy_score(df['late'],preds)

0.9692016992770619